<a href="https://colab.research.google.com/github/longjae/llm_tuning_study/blob/master/gemma_ko_2b_instruction_tuning_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경 설정

## 1-1. 필수 라이브러리 설치

In [35]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U langchain
!pip3 install -q -U langchain-community
!pip3 install -q -U langchain-core

## 1-2. Import Modules

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## 1-3. 저장된 모델 불러오기

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# 로컬에 저장된 파인튜닝된 모델과 토크나이저 로드
model_name = "/content/drive/MyDrive/Colab Notebooks/LLM/gemma-ko-2b-it"
base_model_name = "beomi/gemma-ko-2b"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [41]:
# Hugging face 파이프라인 생성
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)

In [42]:
llm = HuggingFacePipeline(pipeline=pipe)

In [43]:
# 프롬프트 템플릿 작성
prompt = PromptTemplate(template="### Instruction: {question}\n\n### Response:", input_variables=["question"])

# LLMChain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 질문에 대한 답변 생성
question = "최저임금액에 대해 알려줘"
response = llm_chain.run(question)

# 결과 출력
print(response)

### Instruction: 최저임금액에 대해 알려줘

### Response: 최저임금은 근로기준법 제10조에 따라 최저임금위원회가 정하는 최저임금액입니다.


In [44]:
# 시스템 템플릿 작성 - 필요한 기본 설명
system_template = "You are a helpful assistant for law."
# 유저 템플릿 작성 - 사용자 질문에 맞게
user_template = "### Instruction: {question}\n\n### Response:"

# ChatPromptTemplate으로 구성
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", user_template)
])
# LCEL을 이용해 체인 구성
chain = prompt_template | llm

question = "최저임금액에 대해 알려줘"
response = chain.invoke({"question": question})
print(response)

System: You are a helpful assistant for law.
Human: ### Instruction: 최저임금액에 대해 알려줘

### Response: 최저임금액은 1일 8시간 근로에 1일 40시간 근로에 해당하는 시간당 1,000원입니다.


In [48]:
# 시스템 템플릿 작성 - 필요한 기본 설명
system_template = "You are a helpful assistant for law."
# 유저 템플릿 작성 - 사용자 질문에 맞게
user_template = "### Instruction: {question}\n\n### Response:"

# ChatPromptTemplate으로 구성
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", user_template)
])
# LCEL을 이용해 체인 구성
chain = prompt_template | llm

question = "최저임금액에 대해 알려줘"
response = chain.invoke({"question": question})
print(response.split("### Response:")[1])

 최저임금액은 1일 8시간 근로에 1일 40시간 근로에 해당하는 시간당 1,000원입니다.
